<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/026_AgentMemory_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## 🧠 Key Lessons from the Memory + Feedback Agent

### 1. **Agent Memory Must Be Explicitly Managed**

* Agents don’t “remember” unless you **manually append** previous messages (`{"role": "user"}`, `{"role": "assistant"}`) to the `messages` list.
* This is how the LLM gets **context** of what happened before — just like a chat history.
* Use `memory.extend([...])` to grow the context gradually and intentionally.

> ✅ *Memory is not automatic. It is your responsibility to build and manage it.*

---

### 2. **Feedback Loops Enable Iteration**

* Instead of just accepting the LLM’s first summary, the agent pauses and allows for:

  * 👤 User feedback (“make this more concise”)
  * 🔁 LLM revision based on that feedback
* You built an **interactive revision loop** where the user stays in control.

> ✅ *Agents aren’t static — they improve through feedback.*

---

### 3. **Stateful Agents Are More Powerful**

* Because you stored memory and summaries:

  * You could retain what documents were seen
  * You could later use those summaries to write reports, generate comparisons, etc.

> ✅ *Stateful design enables more intelligent, compound workflows.*

---

### 4. **Structured, Concise Prompts Yield Better Results**

* By building prompts that clearly define:

  * The system role (e.g., summarizer)
  * The task (e.g., “summarize this doc”)
  * The content (`content` block)
* You made the LLM **more predictable and accurate**.

> ✅ *Prompt quality directly affects agent performance.*

---

### 5. **Clear Feedback = Better LLM Revisions**

* Your feedback (“make it more succinct and use markdown”) was:

  * Direct
  * Structured
  * Easy for the LLM to interpret and act on
* The revised outputs were significantly better as a result.

> ✅ *LLMs are excellent rewriters — when given good instructions.*

---

## 💾 Knowledge to Retain Going Forward

| Concept                     | Why It Matters                                                        |
| --------------------------- | --------------------------------------------------------------------- |
| **Memory Management**       | Enables agents to work across multiple steps or documents.            |
| **Structured Prompts**      | Improve output consistency and allow for easier parsing.              |
| **Feedback Loops**          | Unlock the ability to refine, correct, and improve LLM output.        |
| **Explicit State Tracking** | Crucial for planning, revisiting, or chaining agent tasks.            |
| **Agent Design Pattern**    | Agents orchestrate tools, respond to feedback, and build upon memory. |



In [1]:
%pip install -qU dotenv openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.0/765.0 kB 35.0 MB/s eta 0:00:00


In [8]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import json
import re
import textwrap

load_dotenv("/content/API_KEYS.env")
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# 🔹 Step 1: Imports and Setup
source_dir = "/content/docs_folder"

# Make sure the directory exists
if not os.path.exists(source_dir):
    raise FileNotFoundError(f"📁 Directory not found: {source_dir}")

# List and build full file paths
file_list = [
    os.path.join(source_dir, f)
    for f in os.listdir(source_dir)
    if os.path.isfile(os.path.join(source_dir, f))
]

# Display the found files
print("📂 Files found:")
for file in file_list:
    print("  -", file)

# 🔹 Step 2: Utility to Read File Preview
def read_file(path, max_chars=1500):
    with open(path, "r", encoding="utf-8") as f:
        return f.read()[:max_chars]

# 🔹 Step 3: Prompt Builder
def build_summary_prompt(content):
    return [
        {
            "role": "system",
            "content": "You are a helpful assistant who summarizes lecture notes clearly and concisely."
        },
        {
            "role": "user",
            "content": f"Please summarize the following document:\n\n{content}"
        }
    ]

# 🔹 Step 4: LLM Call
def generate_response(messages, model="gpt-3.5-turbo"):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=500
    )
    return response.choices[0].message.content.strip()

# 🔹 Step 5: Summary + Feedback Loop
memory = []
summaries = []

for i, file_path in enumerate(file_list):
    filename = os.path.basename(file_path)
    content = read_file(file_path)

    print(f"\n📄 Document {i+1}/{len(file_list)}: {filename}")

    # Start a new summary conversation
    messages = memory + build_summary_prompt(content)
    summary = generate_response(messages)

    print("\n🧠 Summary:\n")
    print(textwrap.fill(summary, width=80))

    while True:
        feedback = input("\n💬 Feedback (or 'next' / 'stop'): ").strip().lower()

        if feedback == "next":
            memory.extend([
                {"role": "user", "content": f"Summarize document: {filename}"},
                {"role": "assistant", "content": summary}
            ])
            summaries.append({"file": filename, "summary": summary})
            break

        elif feedback == "stop":
            print("\n👋 Stopping summary loop.")
            break

        else:
            messages.append({"role": "user", "content": feedback})
            summary = generate_response(messages)
            print("\n🔁 Updated Summary:\n")
            print(textwrap.fill(summary, width=80))

    if feedback == "stop":
        break



📄 Document 1/5: 004_AGENT_Tools.txt

🧠 Summary:

The document discusses the importance of describing tools to an AI agent in an
agentic AI system. It gives an example of automating documentation for Python
code by scanning Python files in a src/ directory and generating documentation
files in a docs/ directory. The importance of defining tools clearly with
naming, parameters, and structured metadata is emphasized. It outlines steps to
define tools with structured metadata in Python and using JSON schema to define
parameters effectively for AI systems.

💬 Feedback (or 'next' / 'stop'): next

📄 Document 2/5: 001_PArse_the Response.txt

🧠 Summary:

The document discusses the process of parsing a response generated by an AI
language model (LLM) to extract the intended action and its parameters. The
response is expected to follow a predefined structure, typically in JSON format
within a markdown code block, for clear parsing and execution. The code snippet
provided demonstrates how to pars


## 🧠 What You’re Learning About Agent Memory

### 1. **Memory Lives Outside the Loop**

```python
memory = []
```

* Memory starts empty and grows across files.
* This mirrors how an agent would accumulate experience or context over a session.
* If you restarted this cell, all memory would reset — it’s up to *you* to persist it if needed (e.g. to disk).

---

### 2. **The Agent's Brain = memory + prompt**

```python
messages = memory + build_summary_prompt(content)
```

* The current prompt is combined with all prior memory to form the **complete input** for the LLM.
* This simulates how a human thinks: "Given everything I already know, here’s what I now need to respond to."

---

### 3. **The Agent Responds to You**

```python
feedback = input("💬 Feedback...")
```

* Your feedback becomes the *new prompt* for the next round.
* The model treats that feedback as if it's part of an ongoing chat — which it is.

---

### 4. **Memory Grows After Every Action**

```python
memory.extend([
    {"role": "user", "content": f"Summarize document: {filename}"},
    {"role": "assistant", "content": summary}
])
```

* You store both the user's task and the model's output as the **basis for future reasoning.**
* This is crucial — the agent can’t remember what it did unless you **manually store it**.

---

### 5. **You (the user) Control the Loop**

```python
if feedback == "stop":
    break
```

* You decide when the session ends.
* This is the earliest version of **agent control flow**, and it will later become programmatic (“terminate” tool, etc.).

---

## 🧩 Key Mental Model:

> You're building a **rolling conversation transcript** that the model uses to make smarter decisions as it goes.

It’s not unlike a court reporter: everything said gets recorded, and later responses are judged in the context of the full record.

---

## ✅ Summary: Why This Code Is Important

| Concept                      | What It Teaches                 |
| ---------------------------- | ------------------------------- |
| `memory = []`                | You own the agent’s memory      |
| `messages = memory + prompt` | Context = memory + task         |
| `extend()` after each step   | Memory grows as the agent works |
| Feedback inside loop         | The agent iterates and improves |
| Exit condition               | Agents need boundaries          |



Understanding the difference between `.append()` and `.extend()` is **key to managing memory correctly** in agent workflows.

---

## 🧠 Difference Between `append()` vs `extend()`

| Method     | What It Does                                                                 |
| ---------- | ---------------------------------------------------------------------------- |
| `append()` | Adds **a single object** to the end of a list.                               |
| `extend()` | Adds **each item from another iterable (like a list)** to the original list. |

---

### 🔍 Let's look at what happens in your code:

```python
memory.extend([
    {"role": "user", "content": f"Summarize document: {filename}"},
    {"role": "assistant", "content": summary}
])
```

This adds **two new dictionary entries** (chat turns) to `memory`. After execution, `memory` will contain:

```python
[
    ...  # all previous chat turns
    {"role": "user", "content": "Summarize document: file_01.txt"},
    {"role": "assistant", "content": "This doc explains memory in LLM agents..."}
]
```

✅ That’s exactly what we want: **two new turns in the conversation**.

---

### ❌ What would go wrong with `append()`?

If you used `.append()` here:

```python
memory.append([
    {"role": "user", "content": "..."},
    {"role": "assistant", "content": "..."}
])
```

Your `memory` list would now contain:

```python
[
    ...,
    [  # a nested list
        {"role": "user", ...},
        {"role": "assistant", ...}
    ]
]
```

👎 That breaks the format. The `messages` list must be **flat**, not nested. The OpenAI API expects a list of dictionaries, not a list of lists.

---

## ✅ Why You Use `append()` for `summaries`

```python
summaries.append({"file": filename, "summary": summary})
```

* In this case, you are adding **one dictionary** to the list of summaries.
* So `.append()` is correct — you're not trying to merge multiple items in.

---

## 📌 Rule of Thumb:

| Use this...              | When you...                                        |
| ------------------------ | -------------------------------------------------- |
| `append(item)`           | Want to add **a single object** to the list        |
| `extend([item1, item2])` | Want to add **multiple objects** from another list |




In [15]:
memory

[{'role': 'user', 'content': 'Summarize document: 004_AGENT_Tools.txt'},
 {'role': 'assistant',
  'content': 'The document discusses the importance of describing tools to an AI agent in an agentic AI system. It gives an example of automating documentation for Python code by scanning Python files in a src/ directory and generating documentation files in a docs/ directory. The importance of defining tools clearly with naming, parameters, and structured metadata is emphasized. It outlines steps to define tools with structured metadata in Python and using JSON schema to define parameters effectively for AI systems.'},
 {'role': 'user', 'content': 'Summarize document: 001_PArse_the Response.txt'},
 {'role': 'assistant',
  'content': 'The document discusses the process of parsing a response generated by an AI language model (LLM) to extract the intended action and its parameters. The response is expected to follow a predefined structure, typically in JSON format within a markdown code block,

In [10]:
print(textwrap.fill(summary, width=80))

The document discusses the importance of updating an agent's memory after
executing an action in an interaction. The agent's memory records user requests,
actions performed, and their outcomes, enabling the agent to retain context for
making informed decisions in future interactions. The code snippet provided
demonstrates how to update the memory with the LLM's response and the result of
the executed action. This process allows the agent to refine its behavior
dynamically as the memory grows and track the status of its work. The final step
in each iteration is to decide whether to continue or terminate based on the
action executed and the task's state.



You’re seeing a mismatch between:

* What **was summarized** ✅
* What got **stored in memory** 🧠
* And what got **added to the summaries list** 📄

Let’s break it down and debug what happened.

---

## 🔍 What’s the Problem?

You summarized **3 files**, but only **2 got stored in memory** and only **1 made it into `summaries`**.

Here’s why:

### ✅ This block adds to memory and summaries:

```python
if feedback == "next":
    memory.extend([
        {"role": "user", "content": f"Summarize document: {filename}"},
        {"role": "assistant", "content": summary}
    ])
    summaries.append({"file": filename, "summary": summary})
    break
```

### ❌ But this block does *not* store anything:

```python
elif feedback == "stop":
    print("\n👋 Stopping summary loop.")
    break
```

So when you hit `"stop"` on the **third document**, the loop exited before saving it to memory or summaries.

---

## ✅ How to Fix It

You want to save the third summary **before** exiting when `"stop"` is triggered.

Here’s the fix:

```python
elif feedback == "stop":
    # ✅ Save the summary before breaking
    memory.extend([
        {"role": "user", "content": f"Summarize document: {filename}"},
        {"role": "assistant", "content": summary}
    ])
    summaries.append({"file": filename, "summary": summary})
    
    print("\n👋 Stopping summary loop.")
    break
```

This way, the **current summary is preserved** before terminating the loop.

---

## ✅ Summary of What You Learned

| Issue               | Why It Happened         | How to Fix                                     |
| ------------------- | ----------------------- | ---------------------------------------------- |
| 3rd summary missing | `"stop"` breaks early   | Add `.extend()` and `.append()` before `break` |
| Memory skipped      | Same reason — no update | Must persist state before exit                 |




In [17]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import json
import re
import textwrap

load_dotenv("/content/API_KEYS.env")
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# 🔹 Step 1: Imports and Setup
source_dir = "/content/docs_folder"

# Make sure the directory exists
if not os.path.exists(source_dir):
    raise FileNotFoundError(f"📁 Directory not found: {source_dir}")

# List and build full file paths
file_list = [
    os.path.join(source_dir, f)
    for f in os.listdir(source_dir)
    if os.path.isfile(os.path.join(source_dir, f))
]

# Display the found files
print("📂 Files found:")
for file in file_list:
    print("  -", file)

# 🔹 Step 2: Utility to Read File Preview
def read_file(path, max_chars=1500):
    with open(path, "r", encoding="utf-8") as f:
        return f.read()[:max_chars]

# 🔹 Step 3: Prompt Builder
def build_summary_prompt(content):
    return [
        {
            "role": "system",
            "content": "You are a helpful assistant who summarizes lecture notes clearly and concisely."
        },
        {
            "role": "user",
            "content": f"Please summarize the following document:\n\n{content}"
        }
    ]

# 🔹 Step 4: LLM Call
def generate_response(messages, model="gpt-3.5-turbo"):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=500
    )
    return response.choices[0].message.content.strip()

# 🔹 Step 5: Summary + Feedback Loop
memory = []
summaries = []

for i, file_path in enumerate(file_list):
    filename = os.path.basename(file_path)
    content = read_file(file_path)

    print(f"\n📄 Document {i+1}/{len(file_list)}: {filename}")

    # Start a new summary conversation
    messages = memory + build_summary_prompt(content)
    summary = generate_response(messages)

    print("\n🧠 Summary:\n")
    print(textwrap.fill(summary, width=80))

    while True:
        feedback = input("\n💬 Feedback (or 'next' / 'stop'): ").strip().lower()

        if feedback == "next":
            memory.extend([
                {"role": "user", "content": f"Summarize document: {filename}"},
                {"role": "assistant", "content": summary}
            ])
            summaries.append({"file": filename, "summary": summary})
            break

        elif feedback == "stop":
            # ✅ Save the summary before breaking
            memory.extend([
                {"role": "user", "content": f"Summarize document: {filename}"},
                {"role": "assistant", "content": summary}
            ])
            summaries.append({"file": filename, "summary": summary})

            print("\n👋 Stopping summary loop.")
            break

        else:
            messages.append({"role": "user", "content": feedback})
            summary = generate_response(messages)
            print("\n🔁 Updated Summary:\n")
            print(textwrap.fill(summary, width=80))

    if feedback == "stop":
        break

📂 Files found:
  - /content/docs_folder/004_AGENT_Tools.txt
  - /content/docs_folder/001_PArse_the Response.txt
  - /content/docs_folder/003_gent Feedback and Memory.txt
  - /content/docs_folder/000_Prompting for Agents -GAIL.txt
  - /content/docs_folder/002_Execute_the_Action.txt

📄 Document 1/5: 004_AGENT_Tools.txt

🧠 Summary:

The document discusses the importance of defining tools for an agentic AI system
and emphasizes the need for clear naming, parameters, and structured metadata.
An example is provided for automating documentation for Python code by listing
files, reading file content, and writing documentation files. The importance of
clearly defining tools to enable effective use by the AI agent is highlighted.
The document also introduces steps to defining tools with structured metadata in
Python and using JSON schema to define parameters.

💬 Feedback (or 'next' / 'stop'): next

📄 Document 2/5: 001_PArse_the Response.txt

🧠 Summary:

The document outlines the process of parsi

In [18]:
# ✅ Print the Final Agent Memory

print("\n🧠 Final Agent Memory:")
for i, msg in enumerate(memory):
    role = msg["role"]
    content = textwrap.fill(msg["content"], width=80)
    print(f"\n{i+1:02d}. {role.upper()}:\n{content}")


🧠 Final Agent Memory:

01. USER:
Summarize document: 004_AGENT_Tools.txt

02. ASSISTANT:
The document discusses the importance of defining tools for an agentic AI system
and emphasizes the need for clear naming, parameters, and structured metadata.
An example is provided for automating documentation for Python code by listing
files, reading file content, and writing documentation files. The importance of
clearly defining tools to enable effective use by the AI agent is highlighted.
The document also introduces steps to defining tools with structured metadata in
Python and using JSON schema to define parameters.

03. USER:
Summarize document: 001_PArse_the Response.txt

04. ASSISTANT:
The document outlines the process of parsing the response generated by the LLM
(Language Model) to extract the intended action and its parameters in a
structured manner. It explains how the response, typically in a predefined
format like JSON within a markdown code block, is parsed to locate the action
ma

In [19]:
# ✅ Print All Summaries

print("\n📚 All Summaries:")
for i, summary_obj in enumerate(summaries):
    filename = summary_obj["file"]
    summary = textwrap.fill(summary_obj["summary"], width=80)

    print(f"\n{i+1:02d}. 📄 {filename}:\n{summary}")



📚 All Summaries:

01. 📄 004_AGENT_Tools.txt:
The document discusses the importance of defining tools for an agentic AI system
and emphasizes the need for clear naming, parameters, and structured metadata.
An example is provided for automating documentation for Python code by listing
files, reading file content, and writing documentation files. The importance of
clearly defining tools to enable effective use by the AI agent is highlighted.
The document also introduces steps to defining tools with structured metadata in
Python and using JSON schema to define parameters.

02. 📄 001_PArse_the Response.txt:
The document outlines the process of parsing the response generated by the LLM
(Language Model) to extract the intended action and its parameters in a
structured manner. It explains how the response, typically in a predefined
format like JSON within a markdown code block, is parsed to locate the action
markers and extract the relevant content. The provided Python code snippet
demonstrat

## Revise the Summaries

In [20]:
# 🔹 Step 5: Summary + Feedback Loop
memory = []
summaries = []

for i, file_path in enumerate(file_list):
    filename = os.path.basename(file_path)
    content = read_file(file_path)

    print(f"\n📄 Document {i+1}/{len(file_list)}: {filename}")

    # Start a new summary conversation
    messages = memory + build_summary_prompt(content)
    summary = generate_response(messages)

    print("\n🧠 Summary:\n")
    print(textwrap.fill(summary, width=80))

    while True:
        feedback = input("\n💬 Feedback (or 'next' / 'stop'): ").strip().lower()

        if feedback == "next":
            memory.extend([
                {"role": "user", "content": f"Summarize document: {filename}"},
                {"role": "assistant", "content": summary}
            ])
            summaries.append({"file": filename, "summary": summary})
            break

        elif feedback == "stop":
            # ✅ Save the summary before breaking
            memory.extend([
                {"role": "user", "content": f"Summarize document: {filename}"},
                {"role": "assistant", "content": summary}
            ])
            summaries.append({"file": filename, "summary": summary})

            print("\n👋 Stopping summary loop.")
            break

        else:
            messages.append({"role": "user", "content": feedback})
            summary = generate_response(messages)
            print("\n🔁 Updated Summary:\n")
            print(textwrap.fill(summary, width=80))

    if feedback == "stop":
        break


📄 Document 1/5: 004_AGENT_Tools.txt

🧠 Summary:

The document discusses the importance of effectively describing tools to an AI
agent for successful interaction. It provides an example of automating
documentation for Python code and emphasizes the need for clear tool
definitions, including naming, parameters, and structured metadata. It outlines
steps for defining tools, such as creating a basic function in Python and using
JSON Schema to define parameters. The goal is to ensure that the AI agent
understands how to utilize the tools provided effectively.

💬 Feedback (or 'next' / 'stop'): Can you make this more succint and ues bullet points and markdown?

🔁 Updated Summary:

- **AI-Agent Tool Descriptions and Naming**:   - Explains the importance of
defining tools for an AI agent.   - Emphasizes the significance of naming,
parameters, and structured metadata. - **Example: Automating Documentation for
Python Code**:   - Illustrates a scenario where an AI agent generates
documentation fo

In [21]:
# ✅ Print the Final Agent Memory

print("\n🧠 Final Agent Memory:")
for i, msg in enumerate(memory):
    role = msg["role"]
    content = textwrap.fill(msg["content"], width=80)
    print(f"\n{i+1:02d}. {role.upper()}:\n{content}")


🧠 Final Agent Memory:

01. USER:
Summarize document: 004_AGENT_Tools.txt

02. ASSISTANT:
- **AI-Agent Tool Descriptions and Naming**:   - Explains the importance of
defining tools for an AI agent.   - Emphasizes the significance of naming,
parameters, and structured metadata. - **Example: Automating Documentation for
Python Code**:   - Illustrates a scenario where an AI agent generates
documentation for Python files.   - Lists necessary functions such as listing
and reading files, and writing documentation. - **Step 1: Defining a Tool with
Structured Metadata**:   - Suggests a basic Python code snippet for listing
Python files in a directory.   - Indicates the necessity for a more structured
description for AI understanding. - **Step 2: Using JSON Schema to Define
Parameters**:   - Describes the utilization of JSON Schema to define tool
parameters succinctly.

03. USER:
Summarize document: 001_PArse_the Response.txt

04. ASSISTANT:
- **Parsing the Response**:   - Extract the intended 

In [22]:
# ✅ Print All Summaries

print("\n📚 All Summaries:")
for i, summary_obj in enumerate(summaries):
    filename = summary_obj["file"]
    summary = textwrap.fill(summary_obj["summary"], width=80)

    print(f"\n{i+1:02d}. 📄 {filename}:\n{summary}")


📚 All Summaries:

01. 📄 004_AGENT_Tools.txt:
- **AI-Agent Tool Descriptions and Naming**:   - Explains the importance of
defining tools for an AI agent.   - Emphasizes the significance of naming,
parameters, and structured metadata. - **Example: Automating Documentation for
Python Code**:   - Illustrates a scenario where an AI agent generates
documentation for Python files.   - Lists necessary functions such as listing
and reading files, and writing documentation. - **Step 1: Defining a Tool with
Structured Metadata**:   - Suggests a basic Python code snippet for listing
Python files in a directory.   - Indicates the necessity for a more structured
description for AI understanding. - **Step 2: Using JSON Schema to Define
Parameters**:   - Describes the utilization of JSON Schema to define tool
parameters succinctly.

02. 📄 001_PArse_the Response.txt:
- **Parsing the Response**:   - Extract the intended action and parameters from
the LLM's output.   - Response should follow a predefine